In [1]:
import os
import shutil

import geopandas as gpd
import pandas as pd
import nivapy3 as nivapy
from shapely.geometry import box

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis()

Connection successful.


## Get main catchment

Also display the bounds as lat/lon.

In [3]:
# Get main catchment
vassom_gdf = nivapy.da.read_postgis(
        'physical',
        'norway_nve_vassdragomrade_poly',
        eng,
)
vassom_gdf = vassom_gdf.query("vassdragsomradenr == '234'")
vassom_gdf.bounds

,minx,miny,maxx,maxy
215,23.701473,68.552719,29.10171,70.750101


The best projected co-ordinate system to use between 24 and 30 degrees E is UTM Zone 35N (see [here](https://epsg.io/32635)). However, most of the datasets I have are national scale and use EPSG 25833, which should be good enough in most cases. Reprojecting to Zone 35N may be wise for any critical area calculations, but I won't worry about this for now.

In [4]:
# Reproject and save
vassom_gdf = vassom_gdf.to_crs(epsg=25833)
fpath = "/home/jovyan/shared/QUANTOM/data/gis/tana_vassdragsomrade.geojson"
vassom_gdf.to_file(fpath, driver="GeoJSON")

## Save bounding box
#bnds = vassom_gdf.bounds
#b = [box(l, b, r, t) for l, b, r, t in zip(bnds.minx, bnds.miny, bnds.maxx, bnds.maxy)]
#vassom_bb = gpd.GeoDataFrame(bnds, geometry=b)
#fpath = "../data/shapefiles/tana_vassdragsomrade_bounding_box.geojson"
#vassom_bb.to_file(fpath, driver="GeoJSON")

/opt/conda/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Extract vector and raster layers of interest

Available vector layers are [here](https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md).

In [5]:
# Path to boundng box
bbox_path = "/home/jovyan/shared/QUANTOM/data/gis/tana_vassdragsomrade.geojson"

# Output folder
out_fold = "/home/jovyan/shared/QUANTOM/data/gis"

# Vector layers of interest
# See https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md
vec_layers = [
    "physical.norway_nibio_corine_2000_poly",
    "physical.norway_nibio_corine_2006_poly",
    "physical.norway_nibio_corine_2012_poly",
    "physical.norway_nibio_corine_2018_poly",
    "physical.norway_nibio_ar50_poly",
    "physical.norway_nve_elvis_river_network_line",
    "physical.norway_nve_innsjo_poly",
]

# Raster layers of interest
# See shared/01_datasets/spatial
ras_layers = [
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif",
    r"/home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif",
]

In [6]:
# Read clip features
bb_gdf = gpd.read_file(bbox_path)

# Loop over layers
for layer in vec_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    schema, table = layer.split(".")
    gdf = nivapy.da.read_postgis(
        schema,
        table,
        eng,
        clip=bb_gdf,
    )

    # Reproject
    gdf = gdf.to_crs(epsg=25833)
    
    # Save
    fpath = os.path.join(out_fold, f"{table}.geojson")
    gdf.to_file(fpath, driver="GeoJSON")

Processing: physical.norway_nibio_corine_2000_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2006_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2012_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_corine_2018_poly
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nibio_ar50_poly
Processing: physical.norway_nve_elvis_river_network_line
Converting to the projection of target dataset (epsg:32633)
Processing: physical.norway_nve_innsjo_poly


In [7]:
# Loop over layers
for layer in ras_layers:
    print(f"Processing: {layer}")

    # Read layer, clipping to bounding box
    fname = os.path.split(layer)[1]
    out_gtiff = os.path.join(out_fold, fname)
    nivapy.spatial.clip_raster_to_gdf(
        layer,
        out_gtiff,
        bb_gdf,
        bounding_box=True,
    )

Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_dem.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})
Processing: /home/jovyan/shared/01_datasets/spatial/norway_kartverket_50m_hillshade.tif
Converting to the projection of target dataset ({'proj': 'utm', 'zone': 33, 'ellps': 'GRS80', 'units': 'm', 'no_defs': True})


## Extract "regine" catchments

Rather than "clipping" these as with the other vector layers above, it makes more sense to just query all the regine units associated with Tana (vassdragsområde 234).

In [8]:
# Get all regine catchments within Tana
reg_gdf = nivapy.da.read_postgis(
        'physical',
        'norway_nve_regine_catchments_poly',
        eng,
)
reg_gdf = reg_gdf[reg_gdf['vassdragsnummer'].str.startswith('234')]

# Save
fpath = "/home/jovyan/shared/QUANTOM/data/gis/tana_regine.geojson"
reg_gdf.to_file(fpath, driver="GeoJSON")

# Sub-catchments

Sub-catchments of interest in Quantom project. Some of these were sampled in autumn 2021 (especially the accessible ones).

In [9]:
shp_path = r"../data/GIS/shapefiles/catchment_boundaries_2021.shp"
sc_gdf = gpd.read_file(shp_path)
sc_gdf.set_index("site_id", inplace=True)
sc_gdf

,vassdragNr,areal_km2,site_name,geometry
site_id,,,,
1,234.GEB,9.40,Njahkajavri,"POLYGON ((850817.528 7669267.751, 851025.000 7..."
4,234.J3A0,84.26,Caskin-jeaggi / Caskinjohka,"POLYGON ((914016.660 7689380.680, 914222.460 7..."
3,234.GDAZ,41.24,Juovvajohka,"POLYGON ((870050.000 7694700.000, 870081.269 7..."
9,234.F3BZ,27.38,Gurrojohka,"POLYGON ((919269.498 7802100.122, 919364.457 7..."
2,234.GEA0,24.86,Vuomajeaggi,"POLYGON ((864315.450 7683072.205, 864387.977 7..."
8,234.F11Z,32.03,Ovddaldasvárri,"POLYGON ((950935.667 7813640.470, 951004.131 7..."
6,234.GAC,51.15,Fáhttevárleakšá,"POLYGON ((884540.333 7756962.415, 884630.715 7..."
7,234.F3C,37.89,Leavvajohka,"POLYGON ((915212.750 7800162.750, 915262.269 7..."
10,234.AAA0,75.94,Jeakkášjávri,"POLYGON ((974096.280 7836925.610, 974160.400 7..."


In [10]:
pts_2021 = ["3c", "4", "6b", "8", "9b", "11", "12", "13", "13b", "14"]
pts_sensors_2022 = ["3c", "9b", "13b", "14"]
sc_gdf = sc_gdf.loc[pts_2021]
sc_gdf
# sensor_sc_df = sc_df.loc[
# sc_gdf.plot(figsize=(12,12))

,vassdragNr,areal_km2,site_name,geometry
site_id,,,,
3c,234.GBCD2Z,15.76,Skierrejohka,"POLYGON ((853805.177 7715715.254, 853880.985 7..."
4,234.J3A0,84.26,Caskin-jeaggi / Caskinjohka,"POLYGON ((914016.660 7689380.680, 914222.460 7..."
6b,234.GBAC,37.32,Raidejohka,"POLYGON ((880244.030 7749880.947, 880334.269 7..."
8,234.F11Z,32.03,Ovddaldasvárri,"POLYGON ((950935.667 7813640.470, 951004.131 7..."
9b,234.F3A1Z,19.13,Darjohka,"POLYGON ((934679.771 7803284.682, 934587.250 7..."
11,234.GCE,43.07,Čuovžajohka,"POLYGON ((880144.957 7674382.026, 880307.077 7..."
12,234.GC2Z,77.82,Šuolggajohka,"POLYGON ((897109.237 7723415.131, 897187.500 7..."
13,234.J10,9.43,Russasjohka,"POLYGON ((923732.330 7703791.862, 923873.435 7..."
13b,234.H5Z,38.35,Mareveadji,"POLYGON ((919812.977 7715112.976, 920075.558 7..."
